#### Seven Bai  白勝文   sevenbai@gmail.com
RAG (Retrieval-Augmented Generation, 檢索增強生成) 是 LLM (Large Language Model 大型語言模型) 非常重要的應用。透過 RAG 技術，LLM 不需要重新訓練就可以回答特定領域的問題，大大提高了 LLM 的實用性。本次分享將帶大家從 text embedding 原理切入，逐步建構一個 RAG 應用，並探討 RAG 的發展趨勢，由淺入深，兩個小時變身 RAG 達人。。

---

### Why RAG?


#### LLM 並不是真的萬事通！

##### LLM 的限制


  - 訓練資料太舊
  - 內部資料不在訓練範圍

##### Fine Tuning

  - 訓練成本高(使用成本低)
  - 不容易更新資料


##### In-Context Learning


  - 使用成本高(沒有訓練成本)
  - 速度慢


##### RAG


  - 使用成本低(沒有訓練成本)
  - 速度快
  - 容易更新資料

### 什麼是 Embedding？

- 文字在多維空間的映射，相近意義的文字在這空間中處於相近的位置

  <img src="images/embedding-1.png" width="400">

- 詞與詞的相對位置也具有相似意義

  <img src="images/embedding-2.png" width="800">

- 甚至可以拿來翻譯

  <img src="images/embedding-3.png" width="600">

- Embedding是用來將抽象的自然語言量化成方便運算的向量

### 使用 OpenAI API 計算 Embedding

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
openaiclient = OpenAI()

In [ ]:
# Show embeddings
embedding = openaiclient.embeddings.create(input = ['今天天氣很好'], model='text-embedding-3-small').data[0].embedding
print(embedding)

In [ ]:
len(embedding)

In [ ]:
import numpy as np

# 計算兩個 embedding 的相似度
def cosine_similarity(a, b):
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
# Calculate distance between two embeddings
while True:
    str1 = input('Enter string1: ')
    if str1 == '': break
    str2 = input('Enter string2: ')
    if str2 == '': break
    data = openaiclient.embeddings.create(input = [str1, str2], model='text-embedding-3-small').data
    similarity = cosine_similarity(data[0].embedding, data[1].embedding)
    print(f'str1: {str1}\nstr2: {str2}\nSimilarity: {similarity}', flush=True)

### 使用 Cohere API 計算 Embedding

In [ ]:
import cohere
co = cohere.Client()

In [ ]:
# Show embeddings
embedding = co.embed(texts=['今天天氣很好'], model='multilingual-22-12').embeddings
print(embedding[0])
len(embedding[0])

In [ ]:
# Calculate distance between two embeddings
while True:
    str1 = input('Enter string1: ')
    if str1 == '': break
    str2 = input('Enter string2: ')
    if str2 == '': break
    embedding = co.embed(texts=[str1, str2], model='multilingual-22-12').embeddings
    similarity = cosine_similarity(embedding[0], embedding[1])
    print(f'str1: {str1}\nstr2: {str2}\nSimilarity: {similarity}', flush=True)

### 使用 Hugging Face 上的 BGE-M3 計算 Embedding

In [ ]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(model_name = model_name, model_kwargs = model_kwargs, encode_kwargs = encode_kwargs)

In [ ]:
# Show embeddings
embedding = hf.embed_query("今天天氣很好")
print(embedding)
len(embedding)

In [ ]:
# Calculate distance between two embeddings
while True:
    str1 = input('Enter string1: ')
    if str1 == '': break
    str2 = input('Enter string2: ')
    if str2 == '': break
    embedding1 = hf.embed_query(str1)
    embedding2 = hf.embed_query(str2)
    similarity = cosine_similarity(embedding1, embedding2)
    print(f'str1: {str1}\nstr2: {str2}\nSimilarity: {similarity}', flush=True)

### 該用哪一家的 Embedding Model？

- Embedding Model 有很多選擇，建議選擇支援多語言版本

  [使用繁體中文評測各家 Embedding 模型的檢索能力](https://ihower.tw/blog/archives/12167)

#### 交叉比對中英日韓文字 embeddings

In [ ]:
import time

allStr = ["我很開心", "I'm very happy", "私は非常に満足している", "나는 매우 행복 해요", "我很傷心", "I'm sad", "私は悲しいです", "나는 슬프다"]

for str1 in allStr:
    for str2 in allStr:
        data = openaiclient.embeddings.create(input = [str1, str2], model='text-embedding-3-small').data
        similarity = cosine_similarity(data[0].embedding, data[1].embedding)
        print(f'str1: {str1}\nstr2: {str2}\nSimilarity: {similarity}', flush=True)
        time.sleep(1.5)
    print('\n')

In [ ]:
import time

allStr = ["我很開心", "I'm very happy", "私は非常に満足している", "나는 매우 행복 해요", "我很傷心", "I'm sad", "私は悲しいです", "나는 슬프다"]

for str1 in allStr:
    for str2 in allStr:
        embedding = co.embed(texts=[str1, str2], model='multilingual-22-12').embeddings
        similarity = cosine_similarity(embedding[0], embedding[1])
        print(f'str1: {str1}\nstr2: {str2}\nSimilarity: {similarity}', flush=True)
        time.sleep(1.5)
    print('\n')


In [ ]:
allStr = ["我很開心", "I'm very happy", "私は非常に満足している", "나는 매우 행복 해요", "我很傷心", "I'm sad", "私は悲しいです", "나는 슬프다"]

for str1 in allStr:
    for str2 in allStr:
        embedding1 = hf.embed_query(str1)
        embedding2 = hf.embed_query(str2)
        similarity = cosine_similarity(embedding1, embedding2)
        print(f'str1: {str1}\nstr2: {str2}\nSimilarity: {similarity}', flush=True)
    print('\n')

<img src="images/multilingual.png" width="1000">